# 5-1 Naive Bayes
# language : Python
https://www.youtube.com/watch?v=BqUmKsfSWho&list=PLqnslRFeH2Upcrywf-u2etjdxxkL8nl7E&index=5

## 0. 모델 기초

### Bayes Theorem
$$ P(A|B) = \frac{P(B|A) \times P(A)}{P(B)} $$


### In our case
$$ P(y|X) = \frac{P(X|y) \times P(y)}{P(X)} $$

### with feature vector X
$$ X = (x_1, x_2, x_3, \ldots ,x_n) $$

### Assume that all features are mutually independent
$$ P(y|X) = \frac{P(x_1|y) \cdot P(x_2|y) \cdots P(x_n|y) \cdot P(y)}{P(X)} $$

### Select class with highest probability
$$ \begin{align*} y &= arg \max_y P(y|X) \\  &= arg \max_y P(x_1|y) \cdot P(x_2|y) \cdots P(x_n|y) \cdot P(y) \\ &= arg \max_y log(P(x_1|y)) + log(P(x_2|y)) + \cdots + log(P(x_n|y)) + log(P(y))\end{align*} $$